In [ ]:
import pandas as pd

In [ ]:
df = pd.read_parquet('dati/dataset_finalissimo.parquet')
materiale_columns = ["('MATERIALE_1', 'sum')", "('MATERIALE_2', 'sum')", "('MATERIALE_3', 'sum')", "('MATERIALE_4', 'sum')"]
df["MATERIALE_TOTAL"] = df[materiale_columns].sum(axis=1)
for col in materiale_columns:
    df[col] = df[col] / df['MATERIALE_TOTAL'] 
df = df.drop(columns=['MATERIALE_TOTAL'])

In [ ]:
min = df["('ANNO_POSA', 'median')"].min()
df["('ANNO_POSA', 'median')"] =  df["('ANNO_POSA', 'median')"] - min

In [ ]:
tipo_columns = ["('TIPO_23', 'sum')", "('TIPO_24', 'sum')", "('TIPO_25', 'sum')"]
df["TIPO_TOTAL"] = df[tipo_columns].sum(axis=1)
for col in tipo_columns:
    df[col] = df[col] / df['TIPO_TOTAL'] 
df = df.drop(columns=['TIPO_TOTAL'])

In [ ]:
df

In [ ]:
def data_processing(df: pd.DataFrame):

    # trasforma il materiale in percentuali da 0 a 1
    materiale_columns = ["('MATERIALE_1', 'sum')", "('MATERIALE_2', 'sum')", "('MATERIALE_3', 'sum')", "('MATERIALE_4', 'sum')"]
    df["MATERIALE_TOTAL"] = df[materiale_columns].sum(axis=1)
    for col in materiale_columns:
        df[col] = df[col] / df['MATERIALE_TOTAL'] 
    df = df.drop(columns=['MATERIALE_TOTAL'])

    # stessa cosa per il tipo
    tipo_columns = ["('TIPO_23', 'sum')", "('TIPO_24', 'sum')", "('TIPO_25', 'sum')"]
    df["TIPO_TOTAL"] = df[tipo_columns].sum(axis=1)
    for col in tipo_columns:
        df[col] = df[col] / df['TIPO_TOTAL'] 
    df = df.drop(columns=['TIPO_TOTAL'])

    #trasforma anno prosa ad una distanza dall anno piu vecchio (1970)
    min = df["('ANNO_POSA', 'median')"].min()
    df["('ANNO_POSA', 'median')"] =  df["('ANNO_POSA', 'median')"] - min
    

In [ ]:
torch.Tensor(df.to_numpy())

In [ ]:
import torch
X_INDECES = [3, 4, 5,6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
Y_INDECES = [10]
len(torch.Tensor(df.to_numpy()))

In [ ]:
from torch.utils.data import Dataset
import pandas as pd
class InReteDataset(Dataset):
    X_INDECES = [3, 4, 5,6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17]
    Y_INDECES = [10]
    def __init__(self, df: pd.DataFrame):
        self.data = torch.Tensor(df.to_numpy())

    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        sample = self.data[idx, X_INDECES]
        label = self.data[idx, Y_INDECES]
    
        return self.data[idx, X_INDECES+Y_INDECES]
        
        
        


In [ ]:
len(dataset)/12

In [ ]:
from gan import GAN

noise_dim = 100  # Dimensionality of the noise vector
data_dim = 16  # Output/input dimensionality (e.g., 28x28 image flattened)
hidden_dim = 128

# Create GAN instance
gan = GAN(noise_dim, data_dim, hidden_dim)

# Generate a batch of noise vectors
batch_size = 12
noise = torch.randn(batch_size, noise_dim)

# Forward pass through the GAN
classification = gan(noise)
#print(classification)

In [ ]:
from torch.utils.data import DataLoader
df = pd.read_parquet('dati/dataset_finalissimo.parquet')

dataset = InReteDataset(df)
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


In [ ]:
from torch import nn
import torch
num_epochs = 100
lr = 0.02
torch.autograd.set_detect_anomaly(True)
# Optimizers
optimizer_g = torch.optim.Adam(gan.generator.parameters(), lr=0.002, weight_decay=0.1)
optimizer_d = torch.optim.Adam(gan.discriminator.parameters(), lr=0.00002, weight_decay=0.1)

# Loss function
criterion = nn.MSELoss()
criterion_disc = nn.BCELoss()
for epoch in range(num_epochs):
    for real_data in dataloader:
        #real_data = torch.cat(real_data, dim=1)

        optimizer_d.zero_grad()

        #generate fake sample
        noise = torch.randn(batch_size, noise_dim)
        fake_data = gan.generator(noise)

        #predict the real images
        distriminator_real_pred = gan.discriminator(real_data)
        #predict the real images
        distriminator_fake_pred = gan.discriminator(fake_data)

        #print(distriminator_fake_pred)
        #calculate loss
        #print(distriminator_real_pred.shape)
        discriminator_loss_real = criterion_disc(distriminator_real_pred,  torch.ones(batch_size, 1))
        discriminator_loss_fake = criterion_disc(distriminator_fake_pred, torch.zeros(batch_size, 1))
        discriminator_loss = discriminator_loss_real + discriminator_loss_fake
        
        #update weights discriminator
        discriminator_loss.backward()
        optimizer_d.step()

        fake_data = gan.generator(noise).detach()

        #predict labels of generated data
        generator_fake_label = gan.discriminator(fake_data)
        
        #calculate loss of generator
        g_loss = criterion_disc(generator_fake_label, torch.ones(real_data.size(0), 1))

        
        optimizer_g.zero_grad()
        g_loss.backward()
        optimizer_g.step()


    # Print progress
    print(f"Epoch [{epoch + 1}/{num_epochs}], Loss D: {discriminator_loss.item():.4f}, Loss G: {g_loss.item():.4f}")


Epoch [1/100], Loss D: 95.3520, Loss G: 1.4121
Epoch [2/100], Loss D: 0.2198, Loss G: 1.6443
Epoch [3/100], Loss D: 0.1888, Loss G: 1.7787
Epoch [4/100], Loss D: 0.1608, Loss G: 1.9368
Epoch [5/100], Loss D: 0.1325, Loss G: 2.1182
Epoch [6/100], Loss D: 0.1084, Loss G: 2.3229
Epoch [7/100], Loss D: 0.0872, Loss G: 2.5284
Epoch [8/100], Loss D: 0.0648, Loss G: 2.8195
Epoch [9/100], Loss D: 0.0552, Loss G: 3.0318
Epoch [10/100], Loss D: 0.0441, Loss G: 3.2738
Epoch [11/100], Loss D: 0.0342, Loss G: 3.5452
Epoch [12/100], Loss D: 0.0294, Loss G: 3.7766
Epoch [13/100], Loss D: 0.0281, Loss G: 3.9212
Epoch [14/100], Loss D: 0.0218, Loss G: 4.2171
Epoch [15/100], Loss D: 0.0161, Loss G: 4.3851
Epoch [16/100], Loss D: 0.0139, Loss G: 4.6554
Epoch [17/100], Loss D: 0.0093, Loss G: 4.8959
Epoch [18/100], Loss D: 0.0153, Loss G: 5.1427
Epoch [19/100], Loss D: 0.0071, Loss G: 5.3079
Epoch [20/100], Loss D: 0.0063, Loss G: 5.4608
Epoch [21/100], Loss D: 0.0051, Loss G: 5.7882
Epoch [22/100], Loss 

In [ ]:
noise = torch.randn(batch_size, noise_dim)
fake_data = gan.generator(noise).detach()
fake_data

In [ ]:
real_data